In [4]:
import os
import json
from typing import TypedDict, Optional, Literal, Dict, Any
from datetime import datetime
from dotenv import load_dotenv
from together import Together
from langgraph.graph import StateGraph

load_dotenv()

True

In [5]:
Together_api_key=os.getenv("TOGETHER_API_KEY")
client=Together()
MODEL_NAME="Qwen/Qwen2.5-72B-Instruct-Turbo"

In [6]:
stage_system_prompt="""
          You are the STAGE-PLANNER AGENT. Your job is to determine the growth stages of any crop based ONLY on the provided soil, water, weather, crop, and sowing-date information. 
          You must NOT invent any weather or soil numbers beyond what is given.

          USER:
          Inputs supplied:
          - location: <LOCATION>
          - crop: <CROP>
          - sowing_date: <YYYY-MM-DD>
          - soil_report_text: <SOIL_REPORT_TEXT>
          - water_report_text: <WATER_REPORT_TEXT>
          - weather_report_text: <WEATHER_REPORT_TEXT>

          TASK:
          Using only the inputs above:
          1) Identify a logical set of growth stages for the given crop.  
          - YOU choose how many stages there should be.  
          - YOU choose the stage names.  
          - Keep stage names simple and agronomically meaningful (e.g., Early Growth, Vegetative, Reproductive, Maturity).  
          - DO NOT use any predefined template or fixed stage list.  
          - Stages must be sequential and cover the full season from sowing to harvest.

          2) Estimate realistic stage durations (days) based on:
          - temperature trend  
          - rainfall / water availability  
          - soil characteristics (WHC, OC, pH, nutrients)  
          - any climate stress patterns  
          These estimates must be REASONABLE, but you are free to infer.

          3) Compute calendar Start and End dates for each stage using the sowing_date.

          4) Provide a confidence score (0.0–1.0) for EACH stage based on clarity of inputs.

          5) DO NOT provide irrigation details, DO NOT provide notes, DO NOT provide nutrient advice.  
          Only give stage name + dates + duration + confidence.

          OUTPUT FORMAT (exact):
          Location: <location>
          Crop: <crop>
          Sowing date: <YYYY-MM-DD>

          Stage plan:
          1) Stage: <stage name>
          Start: <YYYY-MM-DD>
          End: <YYYY-MM-DD>
          Duration (days): <int>
          Confidence: <0.0–1.0>

          2) Stage: <stage name>
          Start: ...
          End: ...
          Duration (days): ...
          Confidence: ...

          (continue until maturity/harvest)

          Assumptions:
          - <list any assumptions or write "None">

          Overall confidence: <0.0–1.0>

          RULES:
          - Do NOT add irrigation recommendations.
          - Do NOT add fertilizer or management notes.
          - Do NOT add anything except the required labelled fields.
          - Use ONLY provided information for reasoning.
          - Keep output clean, concise, and deterministic.
          END.
"""

In [7]:
def stage_planner_agent_together(
    location: str,
    crop: str,
    sowing_date: str,
    soil_report: str,
    water_report: str,
    weather_report: str,
    model_name: str = None,
    temperature: float = 0.2,
    max_tokens: int = 400
) -> str:
    """
    Calls the StagePlanner LLM (Together/Qwen) and returns plain-text stage plan.
    - Keep this function minimal: it only builds prompt, calls model, returns text.
    - Replace `client` and `MODEL_NAME` with your existing client/model variables if different.
    """
    # Use global MODEL_NAME if not passed
    if model_name is None:
        try:
            model_name = MODEL_NAME  # assumed defined in your codebase
        except NameError:
            model_name = "qwen-v1"  # fallback; replace with your actual model

    # escape braces in inputs so .format doesn't break
    def esc(s: str) -> str:
        return (s or "").replace("{", "{{").replace("}", "}}")

    prompt = stage_system_prompt.format(
        location=esc(location),
        crop=esc(crop),
        sowing_date=esc(sowing_date),
        soil_report=esc(soil_report),
        water_report=esc(water_report),
        weather_report=esc(weather_report),
    )

    # --- Together style chat completion (same pattern you used earlier) ---
    try:
        resp = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt},
            ],
            max_tokens=max_tokens,
            temperature=temperature,
        )
        text = resp.choices[0].message.content.strip()
        return text
    except Exception as e:
        # return an explicit error string so downstream can display / log
        return f"Error calling StagePlanner model: {e}"